In [1]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [2]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)

vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
    
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [3]:
np.random.seed(1)
embed_size = 10

# word embeddings
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1

# embedding -> embedding (initially the identity matrix)
recurrent = np.eye(embed_size)

# sentence embedding for empty sentence
start = np.zeros(embed_size)

# embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1

# one hot lookups (for loss function)
one_hot = np.eye(len(vocab))

## 임의의 길이로 순전파 하기
다음 코드는 순전파와 더불어 다음 단어 예측에 대한 논리를 담고 있습니다. 낮설게 느껴지는 구성에도 불구하고, 단위행렬을 이용하는 ㄴ동안 임베딩을 총합하기 위해 따르는 절차는 에전과 동일합니다.  
.  
앞서 말한 바와 같이 이겨에서 단위행렬은 모두 0으로 초기화 되는  recurrent라 불리는 행렬로 대체됩니다. 이 행렬은 학습을 통해 새로운 값을 얻습니다.  
.  
또한 마지막 단어만 예측하는게 아니라 이전 단어까지 생성된 임베딩에 기초해서 다음 단어를 계속 예측해 나갑니다. 이렇게 하면 매번 순전파를 다시 하는 것 보다 효율이 높아집니다.

In [4]:
def predict(sent):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    # forward propagate
    preds = list()
    for target_i in range(len(sent)):

        layer = {}

        # try to predict the next term
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))

        loss += -np.log(layer['pred'][sent[target_i]])

        # generate the next hidden state
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)

    return layers, loss

### *layers = list()
layers 리스트는 순전파를 위한 새로운 방법입니다. 우리가 사용하는 문장(sent)의 길이가 커지면 더 많은 순전파를 해야합니다. 결과적으로 예전처럼 정적 게층 변수를 사용할 수 없습니다. 이제 레이어가 얼마나 필요한지에 따라 이 목록에 새 계층을 붙여나가야 합니다.

## 임의의 길이로 역전파 하기


In [ ]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):  # if not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

In [ ]:
print(layer)

In [ ]:
print(layers)

여기에서 가중 중요한 객체는 두 백터 __layer['state']__ 와 __layer['previous->hidden']__ 을 가지는 리스트 입니다.  
.  
p.296
역전파를 위해 출력 경사도를 취하고 layer['state_delta'] 리스트에 새 객체를 ~... (이해 안됨)

In [8]:
print(layer['state_delta'])

KeyError: 'state_delta'

## 임의의 길이로 가중치 갱신하기

In [5]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])

    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a 
            # later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else:
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())

    # update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))

Perplexity:82.0566976843772
Perplexity:81.94764300940292
Perplexity:81.81416657357991
Perplexity:81.59412969960367
Perplexity:81.17331705643828
Perplexity:80.29954869033996
Perplexity:78.29959197561286
Perplexity:72.674561086639
Perplexity:47.88995969948321
Perplexity:26.52416250957102
Perplexity:20.380986739036135
Perplexity:18.978758195087
Perplexity:17.775942823242435
Perplexity:16.167361954388213
Perplexity:13.734666073001964
Perplexity:10.513795471360048
Perplexity:7.9915610417086445
Perplexity:6.674904932962154
Perplexity:5.833786860862074
Perplexity:5.31184481263934
Perplexity:4.986882111200132
Perplexity:4.769468999586203
Perplexity:4.633500775989438
Perplexity:4.555171751365457
Perplexity:4.498465005274975
Perplexity:4.436391759941303
Perplexity:4.36170951635081
Perplexity:4.278600777153254
Perplexity:4.195052826175365
Perplexity:4.118177548939387


In [6]:
print(layer)

{'pred': array([3.09322309e-03, 5.90908061e-02, 3.41626197e-03, 4.47881817e-03,
       9.51865060e-03, 1.04168006e-02, 6.51265513e-03, 7.22911944e-03,
       1.66065489e-05, 1.19658474e-03, 1.52570415e-02, 7.18232246e-03,
       1.45451566e-07, 1.25677359e-03, 3.75678169e-03, 7.40964444e-17,
       1.05738710e-07, 1.43145683e-35, 1.51321723e-07, 6.82884940e-03,
       2.20939866e-04, 4.80428078e-03, 3.35710562e-03, 4.13364433e-03,
       5.84293785e-03, 8.33966142e-03, 1.91781120e-03, 2.92680579e-20,
       4.34974409e-03, 2.07949560e-03, 4.93524290e-03, 1.08092130e-07,
       1.65655112e-16, 2.31616118e-03, 7.13579325e-17, 3.12836075e-01,
       7.46446061e-03, 3.21406548e-05, 4.95075323e-03, 2.44549903e-03,
       3.84033610e-24, 4.79626081e-03, 8.67148445e-03, 1.05599677e-02,
       9.05985179e-06, 4.23988231e-02, 5.80906812e-03, 2.56995885e-03,
       4.32574283e-03, 1.62065514e-04, 1.44515929e-01, 3.15492164e-03,
       1.34510975e-03, 5.80566417e-04, 3.51810617e-03, 1.52639192e-0

In [9]:
print(layers)

[{'hidden': array([-0.14440977, -0.64322309, -0.38912413,  0.92394088,  0.73134444,
        0.44317634, -1.29727156, -0.06075412, -2.22825832, -0.19572764]), 'hidden_delta': array([-1.8179834 ,  2.84122844,  3.3245737 , -4.73683072, -1.06793371,
        1.735225  ,  1.64472729, -0.24782412,  2.43019301,  1.74300169])}, {'pred': array([0.00250279, 0.00180992, 0.00261685, 0.00265425, 0.00283986,
       0.00296343, 0.00262232, 0.00266671, 0.00248541, 0.00252136,
       0.00270822, 0.00279038, 0.02761   , 0.00265442, 0.00266756,
       0.04700664, 0.0276076 , 0.23882146, 0.026897  , 0.00291577,
       0.00248728, 0.0025921 , 0.00254163, 0.0025273 , 0.00277493,
       0.00282061, 0.00249426, 0.15087278, 0.00282713, 0.00245812,
       0.00269479, 0.02866904, 0.04370904, 0.00258643, 0.04622024,
       0.00158624, 0.00265165, 0.00259994, 0.00265406, 0.00258483,
       0.09710975, 0.00289152, 0.00306267, 0.00305908, 0.00263474,
       0.00182275, 0.00260153, 0.00245802, 0.00269992, 0.00244972,


## 실행과 출력 분석
이 코드를 실행하면 perplexity(혼란도)가 꾸준한 하락세를 보입니다. 이 혼란도는 로그함수를 거쳐 전달된 negated exponented label의 확률입니다(??)  
.  
하지만 혼란도가 이론적으로 나타내는 바는 두 함수의 분포의 차이입니다. 우리 에제에서 완벽한 확룰 분포는 정확한 단어에 대해서는 100% 나머지엔 0%가 될 것 입니다.  
.  
혼란도는 두 함수 분포가 일치하지 않을 때 높고, 일치할 때는 낮습니다(1에 근접). 그러므로 감소하는 혼란도는 좋은것입니다. 뿐만아니라 감소하는 혼란도는 데이터와 일치하는 에측확률을 학습하고 있다는 것을 의미합니다.
.  
그러나 혼란도는 가중치 안에서 무슨 일이 진행되고 있는지는 설명해주지 않습니다. 그래서 혼란도는 지표로써 과용되고 있다는 비판에 직면해 왔습니다.

In [10]:
sent_index = 4

l,_ = predict(words2indices(tokens[sent_index]))

print(tokens[sent_index])

for i,each_layer in enumerate(l[1:-1]):
    input = tokens[sent_index][i]
    true = tokens[sent_index][i+1]
    pred = vocab[each_layer['pred'].argmax()]
    print("Prev Input:" + input + (' ' * (12 - len(input))) +\
          "True:" + true + (" " * (15 - len(true))) + "Pred:" + pred)

['sandra', 'moved', 'to', 'the', 'garden.']
Prev Input:sandra      True:moved          Pred:is
Prev Input:moved       True:to             Pred:to
Prev Input:to          True:the            Pred:the
Prev Input:the         True:garden.        Pred:bedroom.


위 코드는 문장을 취해서 해당 모델이 가장 가능성이 높다고 생각하는 단어를 예측합니다.  
### 예측을 관찰하면 무슨 일이 벌어지는지 이해할 수 있습니다
신경망이 학습하는 과정에서 출력된 예측 결과를 살펴보면 어떤 패턴을 고르는지 뿐만 아니라, 어떤 순서로 학습하는지도 알 수 있습니다. 아래 출력은 신경망이 학습을 100단계 진행한 후의 출력입니다.
![title](https://github.com/limsonghwan/shwan/blob/master/100.jpg?raw=true)
신경망은 무작위 적으로 시작합니다. 위 출력은 첫 무작위 상태에 편향되었을 가능성이 높습니다.  

![title](https://github.com/limsonghwan/shwan/blob/master/10000.jpg?raw=true)
위 출력은 학습을 10000회 후 출력입니다. 신경망은 가장 흔한 단어(the)를 고르고 매 시간마다 'the'로 예측합니다. 이것은 흔한 오류로 순환 신경망이 크게 왜곡된 데이터셋 안의 자세한 세부사항을 익히기 위해선 많은 학습을 거쳐야 합니다.  
![title](https://github.com/limsonghwan/shwan/blob/master/final.jpg?raw=true)
최종 출력입니다. 신경망은 모두 그럴듯한 예측을 해냈습니다. 그러나 신경망이 이 과업을 완벽하게 해결하도록 만드는 방법은 거의 없다는 사실을 알아두는 것이 매우 중요합니다. 사람도 'sandra moved to the'이후에 올 단어를 맞출 수 없습니다. 더 많은 맥락정보가 필요합니다.